In [1]:
import os
import selfask_routines # This script (not in the repository) contains all functions used by the Self Ask pipeline

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document


key_file='../LM_Tests/openai_api_key.txt'
with open(key_file, 'r') as file:
    openai_api_key=file.read()
os.environ["OPENAI_API_KEY"] = openai_api_key

Make text files with timestamps from transcript of Youtube video(s), splitted into langchain Documents of 10 sentences, each with an associated timestamp. Both timestamp and file name ("source") will be used as Document metadata.

In [2]:
texts=[ # Youtube video transcript with timestamps
    'aggregated_f7KSfjv4Oq0.txt', # What Happens If We Throw an Elephant From a Skyscraper? Life & Size 1
    'aggregated_QImCld9YubE.txt', # Why are you alive - Life, Energy and ATP
    'aggregated_QOCaacO8wus.txt', # What is life? Is death real?
    'aggregated_TYPFenJQciw.txt', # The Most Complex Language in the World
    ] 

splits=selfask_routines.make_splits(texts) # Custom text splitting function

print(splits[0])

page_content=' Let's start this video by throwing a
mouse, a dog, and an elephant from a skyscraper onto something soft. Let's say, a stack of mattresses. The mouse lands and is stunned for a moment, before it shakes itself off, and walks away pretty annoyed, because that's a very rude thing to do. The dog breaks all of its bones and dies in an unspectacular way, and the elephant explodes into a red puddle of bones and insides and has no chance to
be annoyed. Why does the mouse survive, but the elephant and dog don't? The answer is size. Size is the most underappreciated 
regulator of living things. Size determines everything about our biology, how we are built, how we
experience the world, how we live and die. It does so because the physical laws are
different for different sized animals.' metadata={'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:00:00]'}


Some examples are constructed upfront to be leverage for few shot prompting.

In [3]:
import fewshots_data # Thi script (not in the repository) contains Self Ask examples, to be used for few shot prompting.

# Make dictionary of few shot examples
few_shot_dict=fewshots_data.few_shot_data()
few_shot_list=[]
for key in few_shot_dict.keys():
    few_shot_list+=few_shot_dict[key]

if len(few_shot_list)==0:
    print(f"ERROR: {len(few_shot_list)=}")

Information relative to the documents can also be useful, and is constructed manually and uploaded as separated Document. Documents containing summaries are also included. These can be both of parts of offline langchain pipelines. They will always be retrieved at every stage.

In [4]:

infos="""Number of videos: 4
Titles: "What is life?", "Life & Size 1", "The Size of Life 2", "Why are you alive - Life, Energy and ATP", "The Most Complex Language in the World"
""" #AC this can stay! There will be metadata simply not inferrable from the text itself, for example if the name of the speaker never appears in what they say themselves in the transcript!
info_doc=Document(page_content=infos,metadata={'source':'documents_info','timestamp':'N/A'})

summ_docs=selfask_routines.make_summ_docs()


The first question demands an intermediate information retrieval, thus will require the usage of Self Ask.

In [5]:
question="How would an animal that deals poorly with water deal with gravity?"
final_answer, chat_history=selfask_routines.single_cycle(question, splits, few_shot_list, info_doc, summ_docs, model='gpt-4o-mini', chat_history=[], verbose=True)

Retrieving most relevant few-shot examplary self ask threads with question...
RETRIEVED THESE SIMILAR QUESTIONS FROM THE FEW SHOT DATABASE:
- What food does the creature that lives here eat?.
- Tell me everything about units of measure.
- What sport does the guy that dress in red play?.


No relevant docs were retrieved using the relevance score threshold 1.0
No relevant docs were retrieved using the relevance score threshold 0.9
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.7000000000000001


Retrieving most relevant few-shot examplary self ask threads with latest follow up question...
RETRIEVED THESE SIMILAR QUESTIONS FROM THE FEW SHOT DATABASE:
- What food does the creature that lives here eat?.
- What sport does the guy that dress in red play?.
- Are both the directors of Jaws and Casino Royale from the same country? 

+ --- --- --- Self Ask cycle: 0 

FOLLOW UP QUESTION: What animal is being referred to that deals poorly with water?
Custom retrieval of documents with latest follow up question...


No relevant docs were retrieved using the relevance score threshold 1.0
No relevant docs were retrieved using the relevance score threshold 0.9
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.7000000000000001
No relevant docs were retrieved using the relevance score threshold 0.6000000000000001


Augmenting with summary documents...
Retrieved 16 documents.
RETRIEVED DOCUMENTS WITH FOLLOW UP QUESTION:
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:02:56]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:04:22]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:02:06]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:05:02]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:03:37]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:00:00]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:00:13]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:01:19]'}
{'source': 'aggregated_QImCld9YubE.txt', 'timestamp': '[00:03:08]'}
{'source': 'aggregated_QImCld9YubE.txt', 'timestamp': '[00:05:11]'}
{'source': 'documents_info', 'timestamp': 'N/A'}
{'source': 'summary_f7KSfjv4Oq0.txt', 'timestamp': 'N/A'}
{'source': 'summary_QImCld9YubE.txt', 'timestamp': 'N/A'}
{'source': 'summary_TYPFenJQciw.txt', 'timestamp'

No relevant docs were retrieved using the relevance score threshold 1.0
No relevant docs were retrieved using the relevance score threshold 0.9
No relevant docs were retrieved using the relevance score threshold 0.8


Augmenting with summary documents...
Retrieved 13 documents.
RETRIEVED DOCUMENTS WITH FOLLOW UP QUESTION:
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:02:06]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:02:56]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:04:22]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:03:37]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:05:02]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:00:13]'}
{'source': 'aggregated_f7KSfjv4Oq0.txt', 'timestamp': '[00:01:19]'}
{'source': 'documents_info', 'timestamp': 'N/A'}
{'source': 'summary_f7KSfjv4Oq0.txt', 'timestamp': 'N/A'}
{'source': 'summary_QImCld9YubE.txt', 'timestamp': 'N/A'}
{'source': 'summary_TYPFenJQciw.txt', 'timestamp': 'N/A'}
{'source': 'summary_MUWUHf-rzks.txt', 'timestamp': 'N/A'}
{'source': 'summary_QOCaacO8wus.txt', 'timestamp': 'N/A'}
Augmenting with documents retrieved with initial question...
Generating few sho

The second question will require the SelfAsk pipeline to acknowledge that summarization tool must be leveraged. This forces the retrieval to focus on the summary documents only.

In [6]:
question="Tell me everything about ATP."
final_answer, chat_history=selfask_routines.single_cycle(question, splits, few_shot_list, info_doc, summ_docs, model='gpt-4o-mini', chat_history=[], verbose=True)

Retrieving most relevant few-shot examplary self ask threads with question...
RETRIEVED THESE SIMILAR QUESTIONS FROM THE FEW SHOT DATABASE:
- Tell me everything about units of measure.
- Tell me about the news about young people.
- Tell me about vehicles produced in Star Wars.


No relevant docs were retrieved using the relevance score threshold 1.0
No relevant docs were retrieved using the relevance score threshold 0.9
No relevant docs were retrieved using the relevance score threshold 0.8


Retrieving most relevant few-shot examplary self ask threads with latest follow up question...
RETRIEVED THESE SIMILAR QUESTIONS FROM THE FEW SHOT DATABASE:
- Tell me everything about units of measure.
- Tell me about the news about young people.
- In how many talks did Jack ask a question?

+ --- --- --- Self Ask cycle: 0 

FOLLOW UP QUESTION: Report the full summary about ATP.
Custom retrieval of documents with latest follow up question...


No relevant docs were retrieved using the relevance score threshold 1.0
No relevant docs were retrieved using the relevance score threshold 0.9
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.7000000000000001


Augmenting with summary documents...
Retrieved 5 documents.
RETRIEVED DOCUMENTS WITH FOLLOW UP QUESTION:
{'source': 'summary_QImCld9YubE.txt', 'timestamp': 'N/A'}
{'source': 'summary_QOCaacO8wus.txt', 'timestamp': 'N/A'}
{'source': 'summary_TYPFenJQciw.txt', 'timestamp': 'N/A'}
{'source': 'summary_MUWUHf-rzks.txt', 'timestamp': 'N/A'}
{'source': 'summary_f7KSfjv4Oq0.txt', 'timestamp': 'N/A'}
Augmenting with documents retrieved with initial question...
Generating few shot prompt...
Using few shot prompt to get intermediate/final answer through direct quoting of the original text...
INTERMEDIATE ANSWER: "ATP is vital for various cellular functions and is universally utilized by all known life forms." | "An ATP molecule is really good for shifting energy around quickly, but it's terrible for storage, since it has only one percent of a glucose molecule's energy at three times its mass." | "It's crucial for almost every process. Plants, fungi, bacteria, and animals need to survive. Without 

Ultimately, the quality of the reasoning provided by this Self Ask pipeline resides in the retrieval of appropriate examples to leverage for the few shot prompt. A larger dataset, as well as an optimal retrieval method from it, are the ultimate key for a successful Q&A. However, the intrinsic capability of improving the pipeline through augmenting the few shot dataset makes it very flexible for multiple use cases.